# Pose Estimation Models
****
#### 주제에 부합한 모델을 찾고 각 모델을 이용한 후처리에 중점을 두었습니다.


## 관점

#### 1. 모델이 person pose를 정확히 detecting할 수 있는가?
- 댄스를 측정하기 위해서 사람을 정확히 인지하는 것은 매우 중요한 문제입니다.
- 탑다운/바텀업 등 여러 방법을 통해 person을 정확히 detecting 해야합니다.
- 당연히 처리 속도의 관점에서는 바텀업방식이 좋겠지만 여러 후처리를 위해서는 정확하게 detecting하는것도 중요해보인다.

#### 2. 모델이 pose estimation을 잘 할 수 있는가?
- 모델들은 2D/3D등 여러 방법으로 detecting한 person의 pose를 estimating합니다.
  또한 관절 좌표를 잘 catch하는것도 이번 프로젝트 주제에 있어 중요한 요소입니다.
- pose estimation또한 single person pose estimation, 또는 multi person pose estimation의 여부에 따라 프로젝트의 확장성에도 영향을 줍니다.


#### 3. 모델이 6fps이상의 realtime을 유지할 수 있는가?
- 모델들은 다양한 방식으로 pose estimating을 진행합니다. 이에따라 realtime을 준수하지 못하는 문제가 발생할 수 있습니다. 이를 해결하기위해 realtime또한 중요한 요소라고 할 수 있겠습니다.
- 또한 이번 프로젝트의 VM환경은 gpu를 지원하지 않는다고 들었기에, cpu에서도 연산할 수 있는 적절한 처리 속도를 요구합니다.

***

## Settings

In [1]:
# !pip install tensorflow opencv-python 
# # https://pysource.com/2019/07/08/yolo-real-time-detection-on-cpu/ 에서 tiny-yolov3 weights,cfg를 가져옵니다.

In [3]:
import tensorflow as tf
import cv2
import numpy as np
import time
from numpy import dot
from numpy.linalg import norm
from matplotlib import pyplot as plt

## [TFLite (movenet/lightning)](https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3)
#### 요약
- single pose detection을 cpu환경에서도 매우 빠르게 잡아낸다.
- 17개 주요 keypoints를 2D coordinate와 confidence로 반환한다.
- 생각보다는 흔들림이 심하고 single pose estimation의 특성상 여러사람이 인식되면 많이 불안정해진다
- 입력값이 제한되어있다.(192x192x3)

In [4]:
# Load model
# https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3에서 tflite를 다운받아온다.
path="put/your/model/path/here"
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

In [8]:
# functions

# draw_keypoints()는 각 프레임별로 받아온 키포인트중 confidence가 일정 수준 이상인
# 키포인트의 위치좌표에 원을 그려준다.
# isMatch의 여부에 따라 다른 색상의 원을 그려주게끔 정의했다.

def draw_keypoints(frame,keypoints,confidence,isMatch):
    y,x,c=frame.shape
    #np.squeeze는 차원을 줄여준다
    shaped=np.squeeze(np.multiply(keypoints,[y,x,1]))
    color = (0,255,0) if isMatch else (255,0,0)
    for kp in shaped:
        ky,kx,kp_conf=kp
        if kp_conf > confidence:
            cv2.circle(frame,(int(kx),int(ky)),4,color,-1)
            # -1은 원을 해당색상으로 채우는걸 의미한다.

# draw_connections()는 각 프레임별로 받아오는 키포인트 중 confidence가 일정 수준 이상인
# 키포인트의 위치좌표들에 대하여 정의된 edge에 맞게끔 선을그어 연결된 뼈대가 보여지게 한다.
# isMatch의 여부에 따라 다른 색상의 선을 그려주게끔 정의했다.
            
def draw_connections(frame,keypoints,edges,confidence,isMatch):
    y,x,c=frame.shape
    #np.squeeze는 차원을 줄여준다
    shaped=np.squeeze(np.multiply(keypoints,[y,x,1]))
    color = (0,255,0) if isMatch else (255,0,0)
    for edge,_ in edges.items():
        p1,p2 = edge
        y1,x1,c1=shaped[p1]
        y2,x2,c2=shaped[p2]
        if(c1>confidence)&(c2>confidence):
            cv2.line(frame,(int(x1),int(y1)),(int(x2),int(y2)),color,2)
            #2는 선 굵기를 의미한다.
   

# cosine_sim()은 테스트 좌표와 정답 좌표의 코사인유사도를 검증하고
# 해당 유사도와 similarity를 비교해 isMatch의 boolean을 결정한다.
# 간단한 측정을 위하여 존재하는 edge들을 벡터로 생각하여
# 모든 벡터가 similarity를 초과하면 True를 반환하게끔 정의하였다
def cosine_sim(my_keypoint,q_keypoint,confidence,similarity):
    my_shape=np.squeeze(my_keypoints)
    q_shape=np.squeeze(q_keypoints)

    result={}
    for edge,_ in EDGES.items():
        p1,p2=edge
        qy1,qx1,qc1=q_shape[p1]
        qy2,qx2,qc2=q_shape[p2]
        a=[qx2-qx1,qy2-qy1]

        my1,mx1,mc1=my_shape[p1]
        my2,mx2,mc2=my_shape[p2]
        b=[mx2-mx1,my2-my1]
        if (qc1>confidence) & (qc2>confidence):
            sim=dot(a,b)/(norm(a)*norm(b))
            if sim<similarity:
                return False
    return True


# make_prediction()은 준비한 모델에 맞게 입력값을 전처리하고,
# invoke를 통해 얻은 결과를 반환해준다.
# 결과 : 각 키포인트를 키로 하는  y , x 좌표값과 confidence 이 담긴 딕셔너리

def make_prediction(img):
    #reshaping img
    img=tf.image.resize_with_pad(np.expand_dims(img,axis=0),192,192)
    input_image=tf.cast(img,dtype=tf.float32)
    
    #setup in/out put
    input_details = interpreter.get_input_details()
    output_details=interpreter.get_output_details()
    
    #make predictions
    interpreter.set_tensor(input_details[0]['index'],np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores=interpreter.get_tensor(output_details[0]['index'])
    #interpreter를 통해 invoke된 결과가 keypoints_with_scores에 업데이트된다.
    
    return keypoints_with_scores

# testing()은 단순히 사진을 테스트해보기위한 용도이며, 해당 사진과 같은 포즈를 취할 때
# isMatch가 true가 되게끔 정의했었다. 다만 쓸일은 거의 없을것이다.
# 정답 키포인트를 반환한다고 생각하면 되겠다.

def testing(img_path='./Desktop/question2.jpg'):
    q_img=cv2.imread(img_path)
    q_keypoints=make_prediction(q_img)
    return q_img,q_keypoints

# thug_life()는 이펙트 확인하기 위해서 정의한 함수이다.
# confidence가 일정 수준 이상인 눈의 좌표를 가져와서 해당 좌표에 맞게끔 이미지를 수정하고
# 최종적으로 프레임에 해당 이미지가 반영되게 해주는 함수이다.

def thug_life(frame,keypoint,confidence,img_path="./Desktop/thug.png",ratio=3):
    y,x,c=frame.shape
    #np.squeeze는 차원을 줄여준다
    shaped=np.squeeze(np.multiply(keypoints,[y,x,1]))
    y0,x0,c0=shaped[0]
    y1,x1,c1=shaped[1]
    y2,x2,c2=shaped[2]
    if (c1>confidence)&(c2>confidence)&(c0>confidence):
        mask = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        
        #*3은 이미지 자체의 크기가 작기에 눈과 눈사이의 거리보다 3배 늘려준것. 수정가능
        resize_x=int(norm([abs(x1-x2),abs(y1-y2)])*ratio)
        resize_y=int(mask.shape[0]*(resize_x/mask.shape[1]))
        center_point=(int((x1+x2-resize_x)/2),int((y1+y2-resize_y)/2))
        
        mask=cv2.resize(mask,(resize_x,resize_y))
        #src, mask, dst는 사이즈가 다 같아야하고, src, dst는 타입도 같아야한다.
        #컬러면 컬러, 그레이스케일이면 그레이스케일로 맞춰주고, mask는 무조건 그레이스케일이여야 한다.
        mask_grey=mask[:,:,3]
        mask_color=mask[:,:,:-1]
        h,w=mask_grey.shape[:2]
        
        #+20은 위치 조절을 위해서 끼워넣었다.
        crop=frame[center_point[1]:center_point[1]+h,center_point[0]:center_point[0]+w]
#         print(mask_color.shape,"\n",mask_grey.shape,'\n',crop.shape)
        cv2.copyTo(mask_color,mask_grey,crop)

In [9]:
# 가장 기본적인 cv2 영상 캡쳐 형식이다.
# cv2에서 지원하는 VideoCapture()를 통해 원하는 영상 path를 넣어주거나
# 0을 넣어 내 로컬 기기의 카메라에 접근할 수 있다.
# 위에서 구현해둔 코드들을 대입하여 여러기능을 확인할 수 있겠다.
# q를 눌리거나 영상이 종료되면 창이 닫힌다.

cap = cv2.VideoCapture(0)
isMatch=False
while cap.isOpened():
    ret,frame=cap.read()
    if ret:
        img=frame.copy()
        keypoints=make_prediction(img)
        #rendering
        confidence_threshold=0.3
#         thug_life(frame,keypoints,confidence_threshold)
#         draw_keypoints(frame,keypoints,confidence_threshold,isMatch)
        cv2.imshow('MoveNet Lightning',frame)
        if cv2.waitKey(10)&0xFF==ord('q'):
            break
    else:
        break
cap.release() #캠끄기
cv2.destroyAllWindows() #창닫기

### 예시영상 pose estimation 결과
#### 당연한 이야기지만 겹쳐지는 순간 single pose estimation에 noise가 크게 발생한다
<img width="80%" src="./img/output1.gif"/>

In [ ]:
# 아래 코드는 임시로 segmentation을 위해 구현해보고 있었던 함수이다.
# 도저히 해당 방법으로는 깔끔하게 segmentation을 진행할 수 없었다..
# 해당 코드에서 Canny를 통해 후처리 이펙트를 넣을 수도 있을것같다는 생각을 했다.

# cap = cv2.VideoCapture(0)

cap = cv2.VideoCapture('./Desktop/4.mp4')
fps = cap.get(cv2.CAP_PROP_FPS) # 프레임 수 구하기
dst=cv2.imread('./Desktop/32.jpeg')

isMatch=True
confidence_threshold=0.4
delay = int(1000/fps)
# 배경 제거 객체 생성 --- ①

knnSubtractor = cv2.createBackgroundSubtractorKNN(100, 32 ,False)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))
kernel_sharpening=np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 배경 제거 마스크 계산 --- ②
    
    img=frame.copy()
    keypoints=make_prediction(img)
    dst=cv2.resize(dst,(frame.shape[1],frame.shape[0]))
#     draw_keypoints(frame,keypoints,confidence_threshold,isMatch)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
#     hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
#     rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
#     yuv=cv2.cvtColor(frame,cv2.COLOR_BGR2YUV)
#     lab=cv2.cvtColor(frame,cv2.COLOR_BGR2LAB)
#     lab2hsv=cv2.cvtColor(lab,cv2.COLOR_BGR2HSV)
#     hsv2lab=cv2.cvtColor(hsv,cv2.COLOR_BGR2LAB)
#     rgb2hsv=cv2.cvtColor(rgb,cv2.COLOR_BGR2HSV)
    
#     blured=cv2.filter2D(frame,-1,kernel_sharpening)
#     blur2rgb=cv2.cvtColor(blured,cv2.COLOR_BGR2RGB)

#     sharpened=cv2.filter2D(frame,-1,kernel_sharpening)
#     sharpened=cv2.filter2D(sharpened,-1,kernel_sharpening)
    blured=cv2.blur(gray,(3,3))
#     sharpened=cv2.filter2D(blured,-1,kernel_sharpening)
    edge=cv2.Canny(blured,300,500)
#     edge=cv2.cvtColor(edge,cv2.COLOR_GRAY2BGR)
    
#     edge=cv2.blur(edge,(7,7))
    background_extraction_mask = fgbg.apply(gray)
#     background_extraction_mask = cv2.morphologyEx(background_extraction_mask, cv2.MORPH_OPEN, kernel)
#     background_extraction_mask = cv2.morphologyEx(background_extraction_mask, cv2.MORPH_OPEN, kernel)

    #     background_extraction_mask = cv2.morphologyEx(background_extraction_mask,cv2.MORPH_CLOSE, kernel)
    background_extraction_mask = cv2.dilate(background_extraction_mask,kernel,iterations=4)

    
    
    background_extraction_mask = np.stack((background_extraction_mask,)*3, axis=-1)
#     draw_keypoints(frame,keypoints,confidence_threshold,isMatch)
#     draw_connections(frame,keypoints,EDGES,confidence_threshold,isMatch)

    
    bitwise_image = cv2.bitwise_and(frame, background_extraction_mask)

    concat_image = np.concatenate((frame,bitwise_image), axis=1)
#     concat_image = np.concatenate((frame,background_extraction_mask), axis=1)
    
#     print(frame.shape,"\n",fgmask.shape,'\n',dst.shape)
#     cv2.copyTo(frame,fgmask,dst)
    cv2.imshow('background extraction video', concat_image)
#     cv2.imshow('back',back)
#     cv2.imshow('fgmask',fgmask)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

의외의 발견 cv2.Canny

![ex_screenshot](./img/Canny.jpg)

## [Mediapipe-MediaPose](https://developers.google.com/mediapipe/solutions/vision/pose_landmarker)
#### 요약
- 탑다운 방식임에도 불구하고 cpu환경에서 빠른 연산을 보여준다 (12~14fps)
- single pose estimating을 진행하여 32개 주요 Landmark를 반환한다.
- pose estimating을 할 때 가려지는 부분들에 대한 예측이 이루어지고 여러사람이 겹치지지 않는 이상 지속적으로 추적을 진행하여 부드러운 movement가 인상적이다.
- 객체가 작아져도 Landmark를 꾸준히 탐색하는 등 single pose estimating을 굉장히 잘 잡아낸다
- 탑다운 방식이다 보니 movenet보다는 프레임드랍이 심하다.

In [12]:
import mediapipe as mp

In [13]:
#functions

# fps_detector()은 프레임의 고정 좌표에 fps값을 표시해주며,
# pTime과 cTime이 꾸준하게 간격을 벌려주게된다.
# 하나의 프레임이 나오는데 걸리는시간의 역수로 초당 프레임(fps)를 측정한다.

def fps_detector(frame,pTime):
    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime
    cv2.putText(frame,str(int(fps)),(70,50),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),3)
    return pTime

In [14]:
#mediapipe의 pose.Pose()모델을 가져오고 이후의 연산에서 입력한 이미지의 process를 진행하여
#landmark나 파라미터의 여부에 따라 segmentation_mask정보를 가져오는등의 여부를 result로 반환한다.
mpPose=mp.solutions.pose
pose=mpPose.Pose()

#mediapipe의 soulution.drawing_utils를 이용해 다양한 수정이 가능하다
mpDraw=mp.solutions.drawing_utils

In [17]:
# 주석처리된 코드는 이미지 저장을 위한 코드입니다.
# 앞선 스켈레톤코드와 다를건없다.
# pose모델에 bgr2rgb처리가 된 이미지를 process하여 results를 반환한다.
# results.pose_landmarks를 통하여 랜드마크의 좌표를 가져올 수 있다.
# 앞선 무브넷과 다르게 confidence는 앞선 코드인 mpPose.Pose()의 파라미터에서 수정가능하다.
# drawing_utils로 간편하게 관절구조를 뽑아낼 수 있다는 장점이 있다.

cap = cv2.VideoCapture('./Desktop/3.mp4')
pTime=0 # for real time
startTime=time.time()

#for writing
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
codec='DIVX'
fourcc=cv2.VideoWriter_fourcc(*codec)
out = cv2.VideoWriter('output3.avi', fourcc, 30.0, (int(width), int(height)))


# while nowTime-startTime<20:
while 1:
    ret, frame = cap.read()
    nowTime=time.time()
#     print(nowTime-startTime)
    if ret:
        rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        if results.pose_landmarks:
            mpDraw.draw_landmarks(frame,results.pose_landmarks,mpPose.POSE_CONNECTIONS)
        pTime=fps_detector(frame,pTime)
        cv2.imshow('image',frame)
        out.write(frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

### 예시영상 pose estimation 결과
#### 무브넷과 다르게 관절이 부드럽고 랜드마크가 많아 더 정밀한 후처리가 가능하다는 점이 있다.
#### 마찬가지로 single pose estimation이지만 한 사람을 추적하는것은 무브넷보다 뛰어나다
#### 하지만 프레임은 확실히 떨어진다
<img width="80%" src="./img/output2.gif"/>

### mediapipe Pose의 강점: segmentation_mask & layout effect
##### 생각보다 강력한 기능인것 같다 무브넷보다 처리도 훨씬 간단하며 프레임드랍도 크지 않다
##### 파라미터의 수정만으로 간단하게 mask를 추출할 수 있다.

In [18]:
# 4가지 parameters가 있지만 크게 영향을 주는 요소는
# model_complexity와 enable_segmentation일것이다.
# model_complexity는 말 그대로 모델의 복잡성을 설정하는것이 0~2(default:1)까지 가능하지만 높을수록 프레임드랍이 크다.
# enable_segmentation은 results에 segmentation_mask 정보를 포함시킬지 여부이다. (default: False)

# segmentation_mask추출을 위해 True로 정의하고 다시 코드를 진행시켜보겠다.
pose_seg=mpPose.Pose(enable_segmentation=True)
mpDraw=mp.solutions.drawing_utils

In [19]:
cap = cv2.VideoCapture('./Desktop/2.mp4')
# cap = cv2.VideoCapture(0)
pTime=0 # for real time
startTime=time.time()
nowTime=time.time()
#for writing
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
codec='DIVX'
fourcc=cv2.VideoWriter_fourcc(*codec)
bg_img=None
# out = cv2.VideoWriter('output3.avi', fourcc, 30.0, (int(width), int(height)))

# while nowTime-startTime<20:
while 1:
    ret, frame = cap.read()
    nowTime=time.time()
#     print(nowTime-startTime)
    if ret:
        if bg_img is None:
            bg_img=cv2.imread('./Desktop/bg2.jpg')
            bg_img=cv2.resize(bg_img,(width,height))
        rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        results = pose_seg.process(rgb)
        if results.segmentation_mask is not None:
            condition= np.stack((results.segmentation_mask,)*3,axis=-1)>0.1
            bg_image=np.zeros(frame.shape,dtype=np.uint8)
            frame=np.where(condition,frame,bg_img)
        
        pTime=fps_detector(frame,pTime)
        cv2.imshow('image',frame)
#         out.write(frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    else:
        break
cap.release()
# out.release()
cv2.destroyAllWindows()

### 예시영상  segmentation 결과
##### single pose estimation의 segmentation확인을 위하여 다른 영상을 가져왔다.
##### detecting이 이루어진 순간부터 정의된 background img로 대체되었다.
##### 조금의 처리가 필요해 보이긴하지만 분명히 강력한 기능이다.

<img width="80%" src="./img/output3.gif"/>

## (추가사항) [YOLOv3-tiny](https://github.com/pjreddie/darknet/blob/master/cfg/yolov3-tiny.cfg)
#### 요약
- 강력한 객체 탐지기능을 자랑하는 yolo의 yolov3의 라이트한 버전이다.
- pose estimating에 특화된 모델은 아니지만 cpu환경에서도 객체탐지의 성능을 높이기위해 만들어진 모델이다
- 만약 human detecting이 기존의 yolo만큼의 성능을 낼 수 있다면, 프레임마다 객체탐지로 human을 detecting하여 mediapipe pose를 적용시킨다면 multi pose estimating이 가능하지 않을까? 라는 생각에서 가져온 모델이다.
- 결과적으로는 tiny라는 이름에 맞게 속도는 빨랐지만 class가 적고, weight도 작아서, 탐지성능이 떨어졌다. mediapipe multi pose detecting은 이루어질 수 없었다...

In [4]:
# yolov3-tiny weights, cfg를 받아와 경로를 입력한다.

net = cv2.dnn.readNet("./Desktop/yolov3-tiny/weights/yolov3-tiny.weights", "./Desktop/yolov3-tiny/cfg/yolov3-tiny.cfg")

# 혹시나해서 기존 yolov3의 weights를 넣어봤지만 제대로 학습되지 않았다.. 
# net = cv2.dnn.readNet("./Desktop/yolov3-tiny/weights/yolov3.weights", "./Desktop/yolov3-tiny/cfg/yolov3-tiny.cfg")

classes = []
with open("./Desktop/yolov3-tiny/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [5]:
# 객체를 탐지함과 동시에 결과값에서 
# 탐지된 결과의 좌표값들을 추출해 cv2.rectangle, text로 객체를 레이블링한다

cap = cv2.VideoCapture('./Desktop/6.mp4')
# cap=cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_PLAIN
starting_time = time.time()
frame_id = 0

#for writing
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
codec='DIVX'
fourcc=cv2.VideoWriter_fourcc(*codec)
# out = cv2.VideoWriter('output6.avi', fourcc, 30.0, (int(width), int(height)))


while 1:
    ret,frame=cap.read()
    if ret:
        frame_id+=1
        height, width, channels=frame.shape
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.1:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = confidences[i]
                color=(0,255,0)
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
                cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, (255,255,255), 3)
        elapsed_time = time.time() - starting_time
        fps = frame_id / elapsed_time
        cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 3, (0, 0, 0), 3)
        cv2.imshow("Image", frame)
#         out.write(frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
                break
    else:
        break
cap.release()
# out.release()
cv2.destroyAllWindows()

### 예시영상  object detection 결과
##### detecting이 빠르긴 하지만 정확도가 상당히 떨어지는 문제가 있다.
##### 이러한 문제는 지속적인 pose estimating에 부정적인 영향을 줄 수 있기에 yolov3-tiny를 응용한 multi pose estimating은 활용되기 힘들것 같다.

<img width="80%" src="./img/output4.gif"/>

***

## 마치며..
#### 기타 상세한 정확도와 시간등 여러 요건들이 이미 명세가 되어있는 pretrained모델을 가져와 학습을 진행했기에 수치적인 요소 보다는 cpu환경에서 얼마나 해당모델이 빠르고 정확하며, 주제에 적합한지 알아보기 위하여 여러 모델들을 적용시켜보았다.
#### 적합한 모델을 찾기위해 꾸준히 학습하고 동향을 살펴보는 것도 좋은 방법이라고 생각하며, 여러가지 상황에 맞는 모델을 찾거나 학습시켜 웹서비스 프로젝트를 좋게 마무리짓고 싶다